In [35]:
import pandas as pd
from tqdm import tqdm

In [36]:
data = pd.read_csv('data_less_ind.csv')

In [37]:
data[data['desc_skills'].isna()]['description']

10        главные новости за 20 июля: итоги торгов. пок...
56       #mail рфпи планирует наращивать партнерство с ...
63       завтра #nlmk нлмк - производственные результат...
76       рывок вверх на возросших оборотах • российский...
101      новости компаний — обзоры прессы перед открыти...
                               ...                        
19208    "five - downgrade jpmorgan снизил рейтинг gdr ...
19215    кто «кормит» севастополь? по оперативным данны...
19216    💥🇷🇺#etln = +3% эталон после новостей по редоми...
19225    "встреча опек+ сегодня в центре внимания. она ...
19233    "#five #ритейлеры #россия сделки "магнита" с "...
Name: description, Length: 1232, dtype: object

In [38]:
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

In [39]:
data.loc[0]['description']

' фокус недели #фн сегодня ммк опубликует финансовую отчётность за 1 кв. 2020 #magn #отчетность магнит представит операционные результаты за 1 кв. 2020 #mgnt #результаты новатэк опубликует финансовую отчётность за 1 кв. 2020 #nvtk #отчетность фрс сша представит решение по процентной ставке #фрс #макро tesla опубликует финансовую отчётность за 1 кв. 2020 #tsla #отчетность microsoft опубликует финансовую отчётность за 1 кв. 2020 #msft #отчетность facebook опубликует финансовую отчётность за 1 кв. 2020 #fb #отчетность 30 апреля детский мир опубликует финансовую отчётность по мсфо за 2019 #dsky #отчетность энел россия опубликует финансовую отчётность по рсбу за 1 кв. #enru #отчетность evraz представит операционные результаты за 1 кв. 2020 #evr #результаты норникель опубликует производственные результаты за 1 кв. 2020 #gmkn #результаты русгидро представит финансовую отчётность по рсбу за 1 кв. 2020 #hydr #отчетность сд нлмк рассмотрит дивиденды за 1 кв. 2020 #nlmk #дивиденды apple опубликуе

In [40]:
data.loc[0]['desc_skills'].split(';')

['magn', 'ммк']

In [41]:
len(data['description'].unique())

15617

In [42]:
for i in data['description'].tolist(): 
    if 'Контроль качества готовой продукции' in i:
        print(i)
        break

In [43]:
helper = data[data['description'] == i]

In [44]:
helper

,description,desc_skills,skills_ind
18003,🇷🇺#mvid #отчетность 12 марта - м.видео - мсфо ...,mvid,3:7


In [49]:
def sorter(ind_list):
    first_ind = []
    for i in ind_list: 
        first_ind.append(i[0])
    sorted_first_ind = sorted(first_ind)
    
    to_return = []
    for i in sorted_first_ind:
        for j in ind_list: 
            if i == j[0]: 
                to_return.append(j)
                break
    return to_return

In [51]:
all_tokens = []
all_masks = []

for i in range(data.shape[0]): 
    tokens = []

    word = 0
    st = ''
    pos = 0
    for j in data.loc[i]['description']:            
        if j.isdigit() or j.isalpha(): 
            word = 1
            st += j 
        elif j == ' ': 
            if word: 
                tokens.append(st)
                st = ''
                word = 0
        elif j == '.':
            if word: tokens.append(st)
            tokens.append('.')
            st = ''
            word = 0
        else: 
            if word: tokens.append(st)
            tokens.append(j)
            st = ''
            word = 0

        pos += 1
        
    
    targ_ind = []
    mark = []
    for j in data.loc[i]['skills_ind'].split(';'):
        helper = j.split(':')
        targ_ind.append([int(helper[0]), int(helper[1])])
    
    targ_ind = sorter(targ_ind)

    k = 0
    pos = 0
    was_before = 0
    for j in tokens: 
        if k + 1 < len(targ_ind) and targ_ind[k][1] <= pos: k += 1

        if j == ' ' and not (targ_ind[k][0] <= pos and pos < targ_ind[k][1]): 
            mark.append(' ')
        elif targ_ind[k][0] <= pos and pos < targ_ind[k][1]: 
            if was_before:
                mark.append('I')
            else: 
                mark.append('B')
                was_before = 1
        else: 
            mark.append('O')
            was_before = 0
        
        pos += len(j)


        
    all_tokens.append('repthing'.join(tokens))
    all_masks.append('*'.join(mark))

In [52]:
data_ready_to_train = pd.DataFrame(columns=['tokens', 'masks'])
data_ready_to_train['tokens'] = all_tokens
data_ready_to_train['masks'] = all_masks

In [53]:
data_ready_to_train

,tokens,masks
0,фокусrepthingнеделиrepthing#repthingфнrepthing...,O*O*O*O*O*B*O*O*O*O*O*O*O*O*O*B*O*O*O*O*O*O*O*...
1,#repthinglsrgrepthingлсрrepthing-repthingопера...,O*B*I*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*...
2,#repthingchmfrepthingсеверстальrepthing(repthi...,O*B*I*O*B*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O
3,"""repthingверсияrepthing:repthingмногоходовочка...",O*O*O*O*O*B*I*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*...
4,🟢repthingновостиrepthingкrepthingэтомуrepthing...,O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*...
...,...,...
17999,smltrepthing+repthing2repthing%repthingнедвига...,B*O*O*O*O*O*O*O*O
18000,"""repthing#repthingnlmkrepthing""repthingноволип...",O*O*B*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*...
18001,заrepthing2020repthingгодrepthingнаrepthingмос...,O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*...
18002,updrepthing:repthingтаrepthingинтерrepthingрао...,O*O*O*O*O*O*O*O


In [54]:
tokens = []

word = 0
st = ''
pos = 0
for j in data.loc[0]['description']:            
    if j.isdigit() or j.isalpha(): 
        word = 1
        st += j 
    elif j == ' ': 
        if word: 
            tokens.append(st)
            st = ''
            word = 0
    elif j == '.':
        if word: tokens.append(st)
        tokens.append('.')
        st = ''
        word = 0
    else: 
        if word: tokens.append(st)
        tokens.append(j)
        st = ''
        word = 0
        
    pos += 1

In [55]:
targ_ind = []
mark = []
for j in data.loc[0]['skills_ind'].split(';'):
    helper = j.split(':')
    targ_ind.append([int(helper[0]), int(helper[1])])

k = 0
pos = 0
was_before = 0
for j in tokens: 
        if k + 1 < len(targ_ind) and targ_ind[k][1] <= pos: k += 1

        if j == ' ' and not (targ_ind[k][0] <= pos and pos < targ_ind[k][1]): 
            mark.append(' ')
        elif targ_ind[k][0] <= pos and pos < targ_ind[k][1]: 
            if was_before:
                mark.append('I')
            else: 
                mark.append('B')
                was_before = 1
        else: 
            mark.append('O')
            was_before = 0
    
        pos += len(j)

In [56]:
len(tokens)

278

In [57]:
len(mark)

278

In [58]:
mark

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O'

In [59]:
it_was_here = 0
for i in range(len(mark)):
    if mark[i] == 'B': 
        if it_was_here != 0: 
            print()
        print(tokens[i], end=' ')
        it_was_here = 1
    elif mark[i] == 'I': 
        print(tokens[i], end=' ')
        it_was_here = 2
    elif it_was_here != 0: 
        print()
        it_was_here = 0

magn 


In [60]:
change_code = data_ready_to_train.copy()

In [61]:
all_correct_mark = []
for i in change_code['masks'].tolist(): 
    correct_mark = []
    for j in i.split('*'):
        if j == 'B': 
            correct_mark.append('B-skill')
        elif j == 'I': 
            correct_mark.append('I-skill')
        elif j == ' ': 
            correct_mark.append(' ')
        else: 
            correct_mark.append('O')
    all_correct_mark.append('*'.join(correct_mark))

In [62]:
maxx = 0
helper = change_code['masks'].tolist()
for i in helper:
    if len(i.split('*')) > maxx: 
        maxx = len(i.split('*'))
print(maxx)

1197


In [63]:
change_code['masks'].tolist()

['O*O*O*O*O*B*O*O*O*O*O*O*O*O*O*B*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O',
 'O*B*I*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O',
 'O*B*I*O*B*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O',
 'O*O*O*O*O*B*I*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*B*O*O*B*O*B*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O',
 'O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*

In [64]:
all_correct_mark

['O*O*O*O*O*B-skill*O*O*O*O*O*O*O*O*O*B-skill*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O',
 'O*B-skill*I-skill*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O',
 'O*B-skill*I-skill*O*B-skill*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O',
 'O*O*O*O*O*B-skill*I-skill*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*B-skill*O*O*B-skill*O*B-skill*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O',
 'O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*

In [65]:
change_code['masks'] = all_correct_mark

In [66]:
change_code.to_csv('data_for_train_new.csv', index=False)

In [32]:
data = pd.read_csv('data_for_train_new.csv')

In [33]:
data

,tokens,masks
0,фокусrepthingнеделиrepthing#repthingфнrepthing...,O*O*O*O*O*B-skill*O*O*O*O*O*O*O*O*O*B-skill*O*...
1,#repthinglsrgrepthingлсрrepthing-repthingопера...,O*B-skill*I-skill*O*O*O*O*O*O*O*O*O*O*O*O*O*O*...
2,#repthingchmfrepthingсеверстальrepthing(repthi...,O*B-skill*I-skill*O*B-skill*O*O*O*O*O*O*O*O*O*...
3,"""repthingверсияrepthing:repthingмногоходовочка...",O*O*O*O*O*B-skill*I-skill*O*O*O*O*O*O*O*O*O*O*...
4,🟢repthingновостиrepthingкrepthingэтомуrepthing...,O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*...
...,...,...
17999,smltrepthing+repthing2repthing%repthingнедвига...,B-skill*O*O*O*O*O*O*O*O
18000,"""repthing#repthingnlmkrepthing""repthingноволип...",O*O*B-skill*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*...
18001,заrepthing2020repthingгодrepthingнаrepthingмос...,O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*...
18002,updrepthing:repthingтаrepthingинтерrepthingрао...,O*O*O*O*O*O*O*O


In [34]:
for i, j in zip(data.loc[1]['tokens'].split('repthing'), data.loc[1]['masks'].split('*')):
    print(i, j)

# O
lsrg B-skill
лср I-skill
- O
операционные O
результаты O
( O
2019г O
) O
* O
продажи O
= O
- O
9 O
% O
до O
84 O
млрд O
руб O
* O
объем O
= O
- O
18 O
% O
до O
817 O
тыс O
. O
кв O
. O
м O
* O
средняя O
цена O
реализации O
= O
+ O
15 O
% O
до O
112 O
тыс O
руб O
/ O
кв O
м O
* O
введение O
в O
эксплуатацию O
чистой O
продаваемой O
площади O
= O
- O
17 O
% O
до O
834 O
тыс O
кв O


In [147]:
data.loc[0]['masks']

'O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*B-skill*I-skill*I-skill*O*O*O*O*O*O*O*O*O*B-skill*I-skill*I-skill*I-skill*I-skill*I-skill*I-skill*O*B-skill*I-skill*I-skill*I-skill*I-skill*I-skill*I-skill*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*B-skill*I-skill*I-skill*I-skill*I-skill*I-skill*I-skill*O*B-skill*O*O*O*O*O*O*O*O*O*O*O*B-skill*O*B-skill*I-skill*I-skill*O*B-skill*I-skill*I-skill*O*O*O*O*B-skill*I-skill*I-skill*O*B-skill*O*B-skill*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O'

In [152]:
masks[0]

'O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*B-skill*I-skill*I-skill*O*O*O*O*O*O*O*O*O*B-skill*I-skill*I-skill*I-skill*I-skill*I-skill*I-skill*O*B-skill*I-skill*I-skill*I-skill*I-skill*I-skill*I-skill*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*B-skill*I-skill*I-skill*I-skill*I-skill*I-skill*I-skill*O*B-skill*O*O*O*O*O*O*O*O*O*O*O*B-skill*O*B-skill*I-skill*I-skill*O*B-skill*I-skill*I-skill*O*O*O*O*B-skill*I-skill*I-skill*O*B-skill*O*B-skill*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O*O'

In [150]:
masks = data['masks'].tolist()

In [151]:
for i in range(len(masks)): 
    masks_help = masks[i].split('*')
    for j in range(len(masks_help)): 
        if j > 0 and masks_help[j] == 'I-skill' and (masks_help[j - 1] == 'O' or masks_help[j - 1] == ' '): 
            print(masks)
            break